<a href="https://colab.research.google.com/github/SHRUTISINHA250714/ML_Models/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other


In [ ]:
from zipfile import ZipFile
dataset="/content/sentiment140.zip"

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
column_names=["target","id","date","flag","user","text"]
twitter_data=pd.read_csv("/content/training.1600000.processed.noemoticon.csv",names=column_names ,encoding="ISO-8859-1")

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
twitter_data['target'].value_counts()


,count
target,
0,800000
1,800000


In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content =re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  # stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words]
  stop_words = set(stopwords.words('english'))
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if word not in stop_words]

  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(X,Y, test_size=0.2 ,stratify=Y, random_state=2)

In [ ]:
# converting textual data to numerical data
vectorizer= TfidfVectorizer()
X_train =vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
# Training LR model
model=LogisticRegression(max_iter=1000)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy= accuracy_score(Y_train,  X_train_prediction)

In [ ]:
print('Accuracy Score on training data: ',training_data_accuracy)

Accuracy Score on training data:  0.79871953125


In [ ]:
X_test_prediction=model.predict(X_test)
test_data_accuracy= accuracy_score(Y_test,  X_test_prediction)

In [ ]:
print('Accuracy Score on test data: ',test_data_accuracy)

Accuracy Score on test data:  0.77668125


**Saving the trained model**

In [ ]:
import pickle

In [ ]:
filename="trained_model.sav"
pickle.dump(model,open(filename,'wb'))

In [ ]:
# using saved model for future prediction
loaded_model= pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
X_new=X_test[200]
print(Y_test[200])
prediction=model.predict( X_new)
print(prediction)
if prediction[0]:
  print("Positive Tweet")
else:
  print("Negative Tweet")

1
[1]
Positive Tweet


In [ ]:
X_new=X_test[3]
print(Y_test[3])
prediction=model.predict( X_new)
print(prediction)
if prediction[0]:
  print("Positive Tweet")
else:
  print("Negative Tweet")

0
[0]
Negative Tweet
